# Bank Marketing Loan Campaign

### Import packages

In [46]:
import pandas as pd
import numpy as np
import pyodbc

### Data CleanUp

In [47]:
# Import Data source form csv file
df = pd.read_csv('bank_marketing.csv')
df

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,41185,56,retired,married,university.degree,no,yes,no,cellular,nov,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,41186,44,technician,married,professional.course,no,no,no,cellular,nov,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [48]:
# Check for empty rows
empty_row = df[df.isna().all(axis=1)]
empty_row

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y


In [49]:
# Check for datatypes
df.dtypes

client_id           int64
age                 int64
job                object
marital            object
education          object
credit_default     object
housing            object
loan               object
contact            object
month              object
day                 int64
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                  object
dtype: object

In [50]:
# Check for unique values, so that can identify it's values and also find how empty cell is defined.
df['credit_default'].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [51]:
# Count of unique values, so that can verify convertion from object datatype to boolean datatype is done properly.
print(df['credit_default'].value_counts()['no'])
print(df['credit_default'].value_counts()['yes'])
print(df['credit_default'].value_counts()['unknown'])

32588
3
8597


In [52]:
# Creating column datatype as boolean by converting yes to true and no to false and keeping unkown as empty
df['credit_default'] = df['credit_default'].map({'yes':True, 'no':False, 'unknown':None}).astype('boolean')

In [53]:
# check for unique values
df['credit_default'].unique()

<BooleanArray>
[False, <NA>, True]
Length: 3, dtype: boolean

In [54]:
# count of unique values
print(df['credit_default'].value_counts()[True])
print(df['credit_default'].value_counts()[False])

3
32588


In [55]:
# check for unique values
print(df['housing'].unique())
print(df['loan'].unique())
print(df['poutcome'].unique())
print(df['y'].unique())

['no' 'yes' 'unknown']
['no' 'yes' 'unknown']
['nonexistent' 'failure' 'success']
['no' 'yes']


In [56]:
# converting to boolean datatype
df['housing'] = df['housing'].map({'yes':True, 'no':False, 'unknown':None}).astype('boolean')
df['loan'] = df['loan'].map({'yes':True, 'no':False, 'unknown':None}).astype('boolean')
df['y'] = df['y'].map({'yes':True, 'no':False, 'unknown':None}).astype('boolean')
df['poutcome'] = df['poutcome'].map({'success':True, 'failure':False, 'nonexistent':None}).astype('boolean')

In [57]:
# check for unique values
print(df['housing'].unique())
print(df['loan'].unique())
print(df['poutcome'].unique())
print(df['y'].unique())

<BooleanArray>
[False, True, <NA>]
Length: 3, dtype: boolean
<BooleanArray>
[False, True, <NA>]
Length: 3, dtype: boolean
<BooleanArray>
[<NA>, False, True]
Length: 3, dtype: boolean
<BooleanArray>
[False, True]
Length: 2, dtype: boolean


In [58]:
# check unique datatype
print(df['job'].unique())
print(df['marital'].unique())
print(df['education'].unique())

['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
['married' 'single' 'divorced' 'unknown']
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']


In [59]:
# Replace unkown with None (empty)
df['job'] = df['job'].replace('unknown', None)
df['marital'] = df['marital'].replace('unknown', None)
df['education'] = df['education'].replace('unknown', None)

# convert datatype to string
df['job'] = df['job'].astype('string')
df['marital'] = df['marital'].astype('string')
df['education'] = df['education'].astype('string')

In [60]:
# check unique datatype
print(df['job'].unique())
print(df['marital'].unique())
print(df['education'].unique())

<StringArray>
[    'housemaid',      'services',        'admin.',   'blue-collar',
    'technician',       'retired',    'management',    'unemployed',
 'self-employed',            <NA>,  'entrepreneur',       'student']
Length: 12, dtype: string
<StringArray>
['married', 'single', 'divorced', <NA>]
Length: 4, dtype: string
<StringArray>
[           'basic.4y',         'high.school',            'basic.6y',
            'basic.9y', 'professional.course',                  <NA>,
   'university.degree',          'illiterate']
Length: 8, dtype: string


In [61]:
# Check if all datatypes are converted properly
df.dtypes

client_id                  int64
age                        int64
job               string[python]
marital           string[python]
education         string[python]
credit_default           boolean
housing                  boolean
loan                     boolean
contact                   object
month                     object
day                        int64
duration                   int64
campaign                   int64
pdays                      int64
previous                   int64
poutcome                 boolean
emp_var_rate             float64
cons_price_idx           float64
cons_conf_idx            float64
euribor3m                float64
nr_employed              float64
y                        boolean
dtype: object

### Splitting into three tables, Client, Economics and Campaign

##### Client

In [94]:
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
# Rename required column(s)
client = client.rename(columns={'client_id': 'id'})
client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,False,False,False
1,1,57,services,married,high.school,<NA>,False,False
2,2,37,services,married,high.school,False,True,False
3,3,40,admin.,married,basic.6y,False,False,False
4,4,56,services,married,high.school,False,False,True
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,False,True,False
41184,41184,46,blue-collar,married,professional.course,False,False,False
41185,41185,56,retired,married,university.degree,False,True,False
41186,41186,44,technician,married,professional.course,False,False,False


##### Campaign

In [63]:
campaign = df[['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'day', 'month']]
campaign = campaign.rename(columns={'campaign':'number_contacts', 'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'poutcome':'previous_outcome', 'y':'campaign_outcome'})
campaign

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,day,month
0,0,1,261,999,0,<NA>,False,13,may
1,1,1,149,999,0,<NA>,False,19,may
2,2,1,226,999,0,<NA>,False,23,may
3,3,1,151,999,0,<NA>,False,27,may
4,4,1,307,999,0,<NA>,False,3,may
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,<NA>,True,30,nov
41184,41184,1,383,999,0,<NA>,False,6,nov
41185,41185,2,189,999,0,<NA>,False,24,nov
41186,41186,1,442,999,0,<NA>,True,17,nov


In [64]:
# Adding new column by combining day and month column and considering the year 2024
campaign['last_contact_date'] = pd.to_datetime(campaign['day'].astype(str) + ' ' + campaign['month'] + ' 2024', format='%d %b %Y')
# Delete column which are not required
campaign.drop(columns=['day', 'month'], inplace=True)
campaign

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,999,0,<NA>,False,2024-05-13
1,1,1,149,999,0,<NA>,False,2024-05-19
2,2,1,226,999,0,<NA>,False,2024-05-23
3,3,1,151,999,0,<NA>,False,2024-05-27
4,4,1,307,999,0,<NA>,False,2024-05-03
...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,<NA>,True,2024-11-30
41184,41184,1,383,999,0,<NA>,False,2024-11-06
41185,41185,2,189,999,0,<NA>,False,2024-11-24
41186,41186,1,442,999,0,<NA>,True,2024-11-17


##### Economics

In [65]:
economics = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics = economics.rename({'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'})
economics

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


##### Export tables to csv file

In [66]:
# Export to csv
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

## Connect to Microsoft SQL (SSMS)

##### Client

In [110]:
# Step 1: Set up the database connection
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=LAPTOP-NP070MIR;"
                      "Database=SAMPLE_DB;"
                      "Trusted_Connection=yes;")

# Create a cursor object to interact with the database
cursor = cnxn.cursor()

# Step 2: Check if the table exists and create it if not
table_name = 'Client'

check_table_sql = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        id INT PRIMARY KEY,
        age INT,
        job VARCHAR(100),
        marital VARCHAR(100),
        education VARCHAR(100),
        credit_default BIT,
        housing BIT,
        loan BIT
    )
END
"""

cursor.execute(check_table_sql)
cnxn.commit()

# Step 3: Insert data from the pandas DataFrame into the table
insert_sql = f"""
INSERT INTO {table_name} (id, age, job, marital, education, credit_default, housing, loan)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

# Insert each row
for row in client.itertuples(index=False, name=None):
    row = list(row)
    row = list(map(lambda x: None if x is pd.NA else x, row))
    row = list(map(lambda x: '1' if x is np.True_ else x, row))
    row = list(map(lambda x: '0' if x is np.False_ else x, row))
    cursor.execute(insert_sql, row)

# Commit the transaction to save changes
cnxn.commit()

# Close the cursor and connection
cursor.close()
cnxn.close()

print("Table created (if not already) and data inserted successfully!")


Table created (if not already) and data inserted successfully!


##### Economics

In [112]:
# Step 1: Set up the database connection
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=LAPTOP-NP070MIR;"
                      "Database=SAMPLE_DB;"
                      "Trusted_Connection=yes;")

# Create a cursor object to interact with the database
cursor = cnxn.cursor()

# Step 2: Check if the table exists and create it if not
table_name = 'Economics'

check_table_sql = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        client_id INT PRIMARY KEY,
        emp_var_rate FLOAT,
        cons_price_idx FLOAT,
        euribor_three_months FLOAT,
        number_employed FLOAT
    )
END
"""

cursor.execute(check_table_sql)
cnxn.commit()

# Step 3: Insert data from the pandas DataFrame into the table
insert_sql = f"""
INSERT INTO {table_name} (client_id, emp_var_rate, cons_price_idx, euribor_three_months, number_employed)
VALUES (?, ?, ?, ?, ?)
"""

# Insert each row
for row in economics.itertuples(index=False, name=None):
    cursor.execute(insert_sql, row)

# Commit the transaction to save changes
cnxn.commit()

# Close the cursor and connection
cursor.close()
cnxn.close()

print("Table created (if not already) and data inserted successfully!")


Table created (if not already) and data inserted successfully!


##### Campaign

In [121]:
# Step 1: Set up the database connection
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=LAPTOP-NP070MIR;"
                      "Database=SAMPLE_DB;"
                      "Trusted_Connection=yes;")

# Create a cursor object to interact with the database
cursor = cnxn.cursor()

# Step 2: Check if the table exists and create it if not
table_name = 'Campaign'

check_table_sql = f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        campaign_id INT IDENTITY(1,1) PRIMARY KEY,  -- Auto-incrementing primary key
        client_id INT,
        number_contacts INT,
        contact_duration INT,
        pdays INT,
        previous_campaign_contacts INT,
        previous_outcome BIT,
        campaign_outcome BIT,
        last_contact_date DATE
    )
END

"""

cursor.execute(check_table_sql)
cnxn.commit()

# Step 3: Insert data from the pandas DataFrame into the table
insert_sql = f"""
INSERT INTO {table_name} (client_id, number_contacts, contact_duration, pdays, previous_campaign_contacts, previous_outcome, campaign_outcome, last_contact_date)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

# Insert each row
for row in campaign.itertuples(index=False, name=None):
    row = list(map(lambda x: None if x is pd.NA else x, row))
    row = list(map(lambda x: '0' if x is np.True_ else x, row))
    row = list(map(lambda x: '1' if x is np.False_ else x, row))
    cursor.execute(insert_sql, row)

# Commit the transaction to save changes
cnxn.commit()

# Close the cursor and connection
cursor.close()
cnxn.close()

print("Table created (if not already) and data inserted successfully!")

Table created (if not already) and data inserted successfully!


##### Connect Power BI with SSMS, build awsome reports/dashboards and enjoy